In [6]:
import datetime
import os
import random
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

import ml_investing_wne.config as config
from ml_investing_wne.data_engineering.load_data import get_hist_data
from ml_investing_wne.data_engineering.prepare_dataset import prepare_processed_dataset
from ml_investing_wne.train_test_val_split import train_test_val_split
from ml_investing_wne.helper import confusion_matrix_plot, compute_profitability_classes, \
    get_final_model_path, add_cost, load_test_dates
from ml_investing_wne.utils import get_logger


seed = 12345
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
                   
logger = get_logger()

In [2]:
df = get_hist_data(currency=config.currency)
df = prepare_processed_dataset(df=df)
X, y, X_val, y_val, X_test, y_test, y_cat, y_val_cat, y_test_cat, train = train_test_val_split(df)

2022-11-01 15:11:57,796 - ml_investing_wne.data_engineering.prepare_dataset - exported to /Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/data/processed/EURCHF/EURCHF_processed_60min.csv
2022-11-01 15:11:57,933 - ml_investing_wne.train_test_val_split - first sequence begins: 2010-01-05 02:00:00
2022-11-01 15:11:57,933 - ml_investing_wne.train_test_val_split - first sequence ends: 2010-01-11 00:00:00
2022-11-01 15:11:58,204 - ml_investing_wne.train_test_val_split - last sequence begins: 2019-12-24 09:00:00
2022-11-01 15:11:58,205 - ml_investing_wne.train_test_val_split - last sequence ends: 2019-12-30 23:00:00
2022-11-01 15:11:59,117 - ml_investing_wne.train_test_val_split - first sequence begins: 2019-12-27 01:00:00
2022-11-01 15:11:59,118 - ml_investing_wne.train_test_val_split - first sequence ends: 2020-01-03 00:00:00
2022-11-01 15:11:59,143 - ml_investing_wne.train_test_val_split - last sequence begins: 2020-12-24 10:00:00
2022-11-01 15:11:5

In [3]:
model_name = get_final_model_path()
model = load_model(model_name)

2022-11-01 15:12:05.821070: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
y_pred = model.predict(X_test)
y_pred_class = y_pred.argmax(axis=-1)

df = add_cost(df)

start_date, end_date = load_test_dates()
lower_bounds = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
upper_bounds = [1 - lower for lower in lower_bounds]

for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
    portfolio_result, hit_ratio, time_active = compute_profitability_classes(df, y_pred, start_date,
                                                                             end_date, lower_bound,
                                                                             upper_bound)

2022-11-01 15:15:19,809 - ml_investing_wne.helper - share_of_time_active for bounds 0.25-0.75 is 0.01 and hit ratio is 0.8750
2022-11-01 15:15:19,809 - ml_investing_wne.helper - share_of_time_active for bounds 0.25-0.75 is 0.01 and hit ratio is 0.8750
2022-11-01 15:15:19,813 - ml_investing_wne.helper - Portfolio result:  101
2022-11-01 15:15:19,813 - ml_investing_wne.helper - Portfolio result:  101
2022-11-01 15:15:22,906 - ml_investing_wne.helper - share_of_time_active for bounds 0.30-0.70 is 0.03 and hit ratio is 0.8438
2022-11-01 15:15:22,906 - ml_investing_wne.helper - share_of_time_active for bounds 0.30-0.70 is 0.03 and hit ratio is 0.8438
2022-11-01 15:15:22,907 - ml_investing_wne.helper - Portfolio result:  102
2022-11-01 15:15:22,907 - ml_investing_wne.helper - Portfolio result:  102
2022-11-01 15:15:25,879 - ml_investing_wne.helper - share_of_time_active for bounds 0.35-0.65 is 0.06 and hit ratio is 0.7938
2022-11-01 15:15:25,879 - ml_investing_wne.helper - share_of_time_acti

In [9]:
lower_bounds = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
upper_bounds = [1 - lower for lower in lower_bounds]
# hours_to_exclude = [datetime.time(21,0,0), datetime.time(22,0,0),  datetime.time(23,0,0)] # 60  - wrong
hours_to_exclude = [datetime.time(20,0,0), datetime.time(21,0,0),  datetime.time(22,0,0)] # 60 - correct, we have to consider that trade is opened at the end of interval for the next hour
# hours_to_exclude = [datetime.time(19,0,0), datetime.time(21,0,0)] # 120 min


for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
    portfolio_result, hit_ratio, time_active = compute_profitability_classes(df, y_pred, start_date,
                                                                             end_date, lower_bound,
                                                                             upper_bound, hours_exclude=hours_to_exclude)

2022-11-01 15:24:14,473 - ml_investing_wne.helper - share_of_time_active for bounds 0.25-0.75 is 0.00 and hit ratio is 1.0000
2022-11-01 15:24:14,473 - ml_investing_wne.helper - share_of_time_active for bounds 0.25-0.75 is 0.00 and hit ratio is 1.0000
2022-11-01 15:24:14,476 - ml_investing_wne.helper - Portfolio result:  99
2022-11-01 15:24:14,476 - ml_investing_wne.helper - Portfolio result:  99
2022-11-01 15:24:17,803 - ml_investing_wne.helper - share_of_time_active for bounds 0.30-0.70 is 0.00 and hit ratio is 0.6667
2022-11-01 15:24:17,803 - ml_investing_wne.helper - share_of_time_active for bounds 0.30-0.70 is 0.00 and hit ratio is 0.6667
2022-11-01 15:24:17,806 - ml_investing_wne.helper - Portfolio result:  99
2022-11-01 15:24:17,806 - ml_investing_wne.helper - Portfolio result:  99
2022-11-01 15:24:21,014 - ml_investing_wne.helper - share_of_time_active for bounds 0.35-0.65 is 0.01 and hit ratio is 0.6500
2022-11-01 15:24:21,014 - ml_investing_wne.helper - share_of_time_active f